# ELM prior generation

In [2]:
import pickle
from pathlib import Path

#import corner
import numpy as np
from scipy import stats

## Define potential and parameters
These are defined in the `elm` package.

In [3]:
from elm import elm

elm.NUM_PARAMS

Using database version X4-2024-12-31 located in: /mnt/home/beyerkyl/x4db/unpack_exfor-2024/X4-2024-12-31


13

In [4]:
from IPython.display import Math, display

for p in elm.params:
    display(Math(f"{p.latex_name} \\text{{ [{ p.unit}]}}"))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Prior distribution
Let's come up with an initial guess following the logic of the [CH89 global optical model potential](https://doi.org/10.1016/0370-1573(91)90039-O) (20 free parameters) which is similar in form to ELM. Because the energy dependencies of the imaginary volume and surface terms are more similar to the [Koning Delaroche](https://www.sciencedirect.com/science/article/pii/S0375947402013210) global optical potential (46 free parameters), we will use that model to inform our prior for $\gamma_W$ and $\gamma_D$. Both of these potentials were re-calibrated with Bayesian uncertainty quantification by [Pruitt, et al.](https://journals.aps.org/prc/abstract/10.1103/PhysRevC.107.014602), and we will take the reported 16, 50, and 84th percentile values for parameters in tables IV and V of that work to construct our prior.  

Of course, we will be fitting to a different corpus of data, and our model is different from both CH89 and KD, so this isn't an exact science. Section 2 of the original CH89 work, as well as a review article by [Hodgson](https://iopscience.iop.org/article/10.1088/0034-4885/34/2/306/meta) and chapter 12 of a textbook by [Satchler](https://www.osti.gov/etdeweb/biblio/5540661) provide physical justifications for model forms and parameter values that we will also consider. 

Finally, it is worthwile to consider the form of 

A particular difference of ELM is the independent isoscalar and isovector geometries. There is no good prior in the literature for isovector geomerties other than the work adjusting the local Koning-Delaroche potentials to quasi-elastic $(p,n)$ by [Danielwicz at al.](https://www.sciencedirect.com/science/article/pii/S0375947416302895). In the absence of strong prior constraints for a global model, we will take the strategy of setting the isoscalar andf isovector geometries to be the same in our prior, and inflate the uncertainties by hand, so that the geometry is essentially only constrained by the cross sections in our Bayesian evidence.

Finally, for the sake of simplicity, we make some simplifying assumptions that depart from modern global potentials. We take the simplest approach for now, but will test later if removing these assumptions (by adding more parameters, and possible more/different evidence) leads to a posterior that is more consistent with the evidence:
 - we fix the geometry of the isoscalar real volume term, the imaginary volume and surface terms to be the same. Typically, the imaginary all terms all have independent geometries, and this is physically justified. 
 - we fix the Coulomb radius to the radius of the isoscalar terms. Typically, the Coulomb radius is fit independently.
 - we will fix the real isoscalar and isovector spin orbit depths to be a constant fraction (0.44) of the corresponding real central depths, following Bohr & Mottelson Ch. 2 (page 238). The imaginary isoscalar spin-orbit depth will be fixed to -3.1 for now, and the isovector spin-orbit depth will be fixeed to 0. Future work including polarized beam elastic and $(p,n)_{IAS}$ data in the corpus will allow us to fit these.
 - it is well known that the optical potential is just the on-shell positive energy sector of single-nucleon self-energy, which is nonlocal in coordinate space, satisifies a dispersion relation, and can be related to experimental observbales both above and below the Fermi energy. We will ignore these important physical constraints for now, choosing a more traditional local, non-dispersive form fit only to positive energy observables (differential cross sections).

In [6]:
from collections import OrderedDict

prior_mean = OrderedDict(
    [
        ("V0", 56.19),  # 51 BM
        ("W0", 9.92),
        ("Wd0", 10.59),
        ("V1", 13.82),  # 33 BM
        ("W1", 0),
        ("Wd1", 27.09),
        ("alpha", -0.36),
        ("gamma_w", 47),
        ("gamma_d", 27),
      #  ("r0", -0.2),
      #  ("r1", -0.2),
        ("r0A", 1.2),
        ("r1A", 1.2),
        ("a0", 0.73),
        ("a1", 0.73),
    ]
)

# diff b/t 16th and 84th pctl in CHUQ - i.e. central 68% confidence interval
prior_ci = OrderedDict(
    [
        ("V0", 1.43 + 1.82),
        ("W0", 4.63 + 2.92),
        ("Wd0", 3.99 + 3.39),
        ("V1", 7.03 + 5.25),
        ("W1", 7.03 + 5.25),  # not in CHUQ or KDUQ, set to same as  uncertainty in V1
        ("Wd1", 12.28 + 8.72),
        ("alpha", 0.03 + 0.02),
        # take this from fitting energy dependence of depth to KDUQ
        ("gamma_w", 30),
        # take this from fitting energy dependence of depth to KDUQ
        ("gamma_d", 3.5),
     #   ("r0", 0.12 + 0.13),
     #   ("r1", 0.12 + 0.13),
        ("r0A", 0.03 + 0.03),
        ("r1A", 0.03 + 0.03),
        ("a0", 0.03 + 0.02),
        ("a1", 0.03 + 0.02),
    ]
)

In [7]:
prior_sigma = OrderedDict([(l, x / 2) for (l, x) in prior_ci.items()])

In [8]:
covariance = np.diag(list(prior_sigma.values())) ** 2
mean = np.array(list(prior_mean.values()))

In [9]:
n_prior_samples = 100000
prior_distribution = stats.multivariate_normal(mean, covariance)
prior_samples = prior_distribution.rvs(size=n_prior_samples)

In [10]:
with open("prior_distribution.pickle", "wb") as f:
    pickle.dump(prior_distribution, f)